# Overview

Philosophically, A **Trial** is sample take from the Search Space. Said another way, a Trial is an observation of one of the possible scenarios defined as part of the Search Space.

Under the hood, the Trials object is what controls how the Hyperopt framework iterates over the Search Space and selects the next set of hyperparameters and produces the next Trial object.

There are three types of Trial objects: Trials, MongoTrials, SparkTrials. We will look at the "base" Trials object here. We will look at the SparkTrials object when we cover Apache Spark in the Notebook regarding [Spark Integration](Hyperopt%20Spark%20Integration.ipynb). We will not look at the MondoTrials object as I am not yet (and at this point do not plan to start) using MongoDB.

In [50]:
import hyperopt

# Define the search space
space = hyperopt.hp.choice('my_choice', [
    {
        'name': 'model a',
        'x': hyperopt.hp.choice('model_a_x', [1,2,4,6,8])
    },
    {
        'name': 'model b',
        'x': hyperopt.hp.choice('model_b_x', [0,1,2,3,4]),
        'y': hyperopt.hp.choice('model_b_y', [0,3,5,7,9])        
    }
])

We can have a look at a single sample

In [51]:
print(hyperopt.pyll.stochastic.sample(space))

{'name': 'model a', 'x': 8}


We can define a loss function and search through our parameter space for the optimal value using the hyperopt framework:

In [52]:
import hyperopt
import numpy

# Define the objective function
def objective(args):
    x = args['x']
    y = args['y'] if 'y' in args.keys() else 0
    return x + y

# Define an object to keep track of the "trials" in the search path
trials = hyperopt.Trials()
    
# Optimize the search space and retrieve the index which points to the best points in the search space
optimal_args_index = hyperopt.fmin(objective, space, algo=hyperopt.tpe.suggest, max_evals=10, trials=trials, rstate=numpy.random.default_rng(42))
    
# Retrieve the resulting hyperparameter set from the search space using the index
optimal_hyperparams = hyperopt.space_eval(space, optimal_args_index)

# Print the results
print("=========================")
print("Optimal args index:")
print(optimal_args_index)
print("Best hyperparameters:")
print(optimal_hyperparams)

100%|██████████| 10/10 [00:00<00:00, 114.13trial/s, best loss: 1.0]
Optimal args index:
{'model_a_x': 0, 'my_choice': 0}
Best hyperparameters:
{'name': 'model a', 'x': 1}


We can see that "model a" was selected as it yields the minimal results from the objective function.

**Note:** We see that the search algorithm has a lot of repetitions... This is because it the only boundary on the search is the max_evals parameter. We will look at optimizing the search algorithm later on. For example when we utilize the loss_threshold to allow for early termination.

Having a look at the trials opject we inspect it's type and the useful properties.

In [4]:
trials

In [15]:
dir(trials)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_dynamic_trials',
 '_exp_key',
 '_ids',
 '_insert_trial_docs',
 '_trials',
 'aname',
 'argmin',
 'assert_valid_trial',
 'asynchronous',
 'attachments',
 'average_best_error',
 'best_trial',
 'count_by_state_synced',
 'count_by_state_unsynced',
 'delete_all',
 'fmin',
 'idxs',
 'idxs_vals',
 'insert_trial_doc',
 'insert_trial_docs',
 'losses',
 'miscs',
 'new_trial_docs',
 'new_trial_ids',
 'refresh',
 'results',
 'source_trial_docs',
 'specs',
 'statuses',
 'tids',
 'trial_attachments',
 'trials',
 'vals',
 'view']

# Common Analysis Tasks
In the next section we look at code examples of extracting valuable information from the Trials object.

## Get Trial Metadata

In [53]:
trials.trials[0:2]

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': 8.0, 'status': 'ok'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'model_a_x': [0],
    'model_b_x': [],
    'model_b_y': [],
    'my_choice': [0]},
   'vals': {'model_a_x': [4],
    'model_b_x': [],
    'model_b_y': [],
    'my_choice': [0]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2021, 12, 16, 21, 43, 51, 150000),
  'refresh_time': datetime.datetime(2021, 12, 16, 21, 43, 51, 151000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': 4.0, 'status': 'ok'},
  'misc': {'tid': 1,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'model_a_x': [],
    'model_b_x': [1],
    'model_b_y': [1],
    'my_choice': [1]},
   'vals': {'model_a_x': [],
    'model_b_x': [1],
    'model_b_y': [1],
    'my_choice': [1]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': d

## Get Best Trial Metadata
We can see the best trial:

In [54]:
trials.best_trial

{'state': 2,
 'tid': 2,
 'spec': None,
 'result': {'loss': 1.0, 'status': 'ok'},
 'misc': {'tid': 2,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'model_a_x': [2],
   'model_b_x': [],
   'model_b_y': [],
   'my_choice': [2]},
  'vals': {'model_a_x': [0],
   'model_b_x': [],
   'model_b_y': [],
   'my_choice': [0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2021, 12, 16, 21, 43, 51, 166000),
 'refresh_time': datetime.datetime(2021, 12, 16, 21, 43, 51, 166000)}

## Get Trial ID

In [55]:
trials.trials[0]["tid"]

0

## Get Hyperparameters For Best Trial

In [56]:
trials.argmin

{'model_a_x': 0, 'my_choice': 0}

In [57]:
hyperopt.space_eval(space, trials.argmin)

{'name': 'model a', 'x': 1}

## Get Hyperparameters For Arbitrary Trial

In [58]:
trials.trials[2]["misc"]["vals"]

{'model_a_x': [0], 'model_b_x': [], 'model_b_y': [], 'my_choice': [0]}

We need to convert this value into another form. The [source code](https://github.com/hyperopt/hyperopt/blob/0b49cde7c0860542b17e8f6102dcf46af4739d23/hyperopt/base.py#L619) for doing this is in the *argmin* property on the Trials object

In [67]:
vals = trials.trials[2]["misc"]["vals"]

def correct_vals_object_format(vals):
    rval = {}
    for k, v in list(vals.items()):
        if v:
            rval[k] = v[0]
    return rval
        
corrected_vals = correct_vals_object_format(vals)
corrected_vals

{'model_a_x': 0, 'my_choice': 0}

Once converted, we can use the same *space_eval* function to derive the hyperparameters.

In [68]:
hyperopt.space_eval(space, corrected_vals)

{'name': 'model a', 'x': 1}

## Get Scores For A Trial

In [73]:
trials.results[0]

{'loss': 8.0, 'status': 'ok'}